In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np

In [3]:
actions=np.array(['one','two','three'])
no_sequences=30
sequence_length=30

In [4]:
label_map = {label:num for num, label in enumerate(actions)}

In [5]:
label_map

{'one': 0, 'three': 2, 'two': 1}

In [6]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            path="/content/drive/MyDrive/Gesture_recognition/Dataset/"+action+"/"+str(sequence)+"/{}.npy".format(frame_num)
            res = np.load(path)
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [8]:
X=np.array(sequences)
X.shape
len(X)

90

In [9]:
y=to_categorical(labels).astype(int)

In [18]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.05)

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [13]:
log_path="/content/drive/MyDrive/Gesture_recognition/Logs"
tb_callback = TensorBoard(log_dir=log_path)

In [14]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(LSTM(128, return_sequences=False, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            48896     
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 64)            33024     
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 32)                4128      
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 51        
Total params: 185,443
Trainable params: 185,443
Non-trainable params: 0
__________________________________________________

In [16]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [21]:
model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback])

Epoch 1/500
3/3 [==============================] - 2s 887ms/step - loss: 0.7726 - categorical_accuracy: 0.7294
Epoch 2/500
3/3 [==============================] - 1s 290ms/step - loss: 0.7487 - categorical_accuracy: 0.7647
Epoch 3/500
3/3 [==============================] - 1s 281ms/step - loss: 0.7029 - categorical_accuracy: 0.7765
Epoch 4/500
3/3 [==============================] - 1s 277ms/step - loss: 0.6427 - categorical_accuracy: 0.7765
Epoch 5/500
3/3 [==============================] - 1s 271ms/step - loss: 0.5867 - categorical_accuracy: 0.7765
Epoch 6/500
3/3 [==============================] - 1s 268ms/step - loss: 0.5072 - categorical_accuracy: 0.8118
Epoch 7/500
3/3 [==============================] - 1s 262ms/step - loss: 0.7330 - categorical_accuracy: 0.7882
Epoch 8/500
3/3 [==============================] - 1s 293ms/step - loss: 0.6372 - categorical_accuracy: 0.7529
Epoch 9/500
3/3 [==============================] - 1s 271ms/step - loss: 0.6171 - categorical_accuracy: 0.7529
E

KeyboardInterrupt: ignored

In [23]:
model.save('action.h5')

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)

array([[[4, 0],
        [1, 0]],

       [[3, 2],
        [0, 0]],

       [[1, 0],
        [1, 3]]])

In [ ]:
accuracy_score(ytrue, yhat)

0.6